<a href="https://colab.research.google.com/github/manishyerram89/Image-Segmentation/blob/main/ImageSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate

In [ ]:
def generate_synthetic_data(num_samples, image_size=(256, 256)):
    x_data = np.random.rand(num_samples, *image_size, 3)
    y_data = (x_data.mean(axis=3, keepdims=True) > 0.5).astype(np.float32)  # Generate binary masks
    return x_data, y_data


In [ ]:
def unet(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)

    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = unet(input_shape=(256, 256, 3))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
x_train, y_train = generate_synthetic_data(num_samples=100)

In [7]:
model.fit(x_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
7/7 [==============================] - 671s 93s/step - loss: 0.6704 - accuracy: 0.6997
Epoch 2/10
7/7 [==============================] - 658s 92s/step - loss: 0.5356 - accuracy: 0.8279
Epoch 3/10
7/7 [==============================] - 692s 98s/step - loss: 0.3424 - accuracy: 0.8843
Epoch 4/10
7/7 [==============================] - 666s 93s/step - loss: 0.2127 - accuracy: 0.9208
Epoch 5/10
7/7 [==============================] - 669s 93s/step - loss: 0.1519 - accuracy: 0.9379
Epoch 6/10
7/7 [==============================] - 675s 94s/step - loss: 0.1277 - accuracy: 0.9451
Epoch 7/10
7/7 [==============================] - 657s 92s/step - loss: 0.1070 - accuracy: 0.9546
Epoch 8/10
7/7 [==============================] - 662s 93s/step - loss: 0.0904 - accuracy: 0.9627
Epoch 9/10
7/7 [==============================] - 662s 93s/step - loss: 0.0782 - accuracy: 0.9685
Epoch 10/10
7/7 [==============================] - 672s 95s/step - loss: 0.1543 - accuracy: 0.9342


In [8]:
new_image = np.random.rand(1, 256, 256, 3)

In [9]:
predictions = model.predict(new_image)

1/1 [==============================] - 2s 2s/step


In [10]:
print(new_image)

[[[[0.5913353  0.48430542 0.97278827]
   [0.30369811 0.12285115 0.86421584]
   [0.62657885 0.77892585 0.28458983]
   ...
   [0.9964885  0.96854095 0.83609584]
   [0.01798325 0.76837449 0.47409125]
   [0.04526025 0.9636595  0.69763347]]

  [[0.41536611 0.20980208 0.82788207]
   [0.7129057  0.74466776 0.78282083]
   [0.47516213 0.27501827 0.85964017]
   ...
   [0.03433484 0.72382351 0.97266697]
   [0.01166037 0.62637526 0.08004548]
   [0.68792633 0.4550195  0.57716101]]

  [[0.64953038 0.11261509 0.40806719]
   [0.00330702 0.72933671 0.3662195 ]
   [0.38338745 0.40564342 0.87368549]
   ...
   [0.31567107 0.97675922 0.3649939 ]
   [0.03693257 0.93322284 0.6641617 ]
   [0.42890582 0.24214514 0.83257257]]

  ...

  [[0.45034227 0.45045544 0.15898564]
   [0.85893293 0.70868692 0.93211229]
   [0.59499423 0.48479865 0.64308001]
   ...
   [0.81442752 0.87318521 0.09812909]
   [0.63185218 0.15407205 0.64000853]
   [0.21105676 0.92940217 0.1530463 ]]

  [[0.6733869  0.16423742 0.44474578]
   [0.0

In [ ]:



print(new_image)


Epoch 1/10
7/7 [==============================] - 421s 58s/step - loss: 0.6690 - accuracy: 0.6709
Epoch 2/10
7/7 [==============================] - 408s 57s/step - loss: 0.5271 - accuracy: 0.8256
Epoch 3/10
7/7 [==============================] - 407s 57s/step - loss: 0.3254 - accuracy: 0.8880
Epoch 4/10
7/7 [==============================] - 408s 57s/step - loss: 0.1958 - accuracy: 0.9244
Epoch 5/10
7/7 [==============================] - 408s 57s/step - loss: 0.1474 - accuracy: 0.9376
Epoch 6/10
7/7 [==============================] - 408s 57s/step - loss: 0.1236 - accuracy: 0.9465
Epoch 7/10
7/7 [==============================] - 407s 57s/step - loss: 0.0982 - accuracy: 0.9591
Epoch 8/10
7/7 [==============================] - 408s 57s/step - loss: 0.1156 - accuracy: 0.9486
Epoch 9/10
7/7 [==============================] - 407s 57s/step - loss: 0.0908 - accuracy: 0.9614
Epoch 10/10
1/1 [==============================] - 1s 1s/step
[[[[0.80614845 0.00263867 0.64478662]
   [0.97283788 0.0